In [148]:
import numpy as np
import pandas as pd

color_est = pd.read_csv("../data/color2emotion_rescaling.csv",index_col = False)
emotion_est = pd.read_csv("../result/emotion_estimation_hand_craft.csv")
action_est = pd.read_csv("../result/action_estimation.csv")

In [149]:
#color_est

In [150]:
emotion_est


,감정,에너지,회복,순환,정화
0,행복,0.300379,0.469413,0.287386,0.282726
1,열정,1.000000,0.598599,0.395890,0.400756
2,두려움,0.140877,0.477591,0.057623,0.300837
3,분노,0.364110,0.405571,0.323333,0.367194
4,우울,0.341376,1.000000,0.250951,0.360276
5,슬픔,0.191463,0.461462,0.185052,0.320086


In [151]:
action_est

,감정,에너지,회복,순환,정화
0,명상,0.400750,0.455728,0.589044,0.325155
1,운동,0.364873,0.483403,0.383764,0.324081
2,산책,0.320345,0.461215,0.271852,0.328704
3,수면,0.281741,0.407013,0.238239,0.359288
4,독서,0.326346,0.485431,0.260725,0.255343


In [152]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from scipy.stats import pearsonr


def pearson_similarity(a, b):
  return np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))

In [181]:
# 감정 -> 효능 <- 해동 시나리오
class emotion2item():
    def __init__(self ,action_df, emotion_df):
        self.action_df = action_df
        self.emothon_df = emotion_df
             
        
    def sim_item_MAE(self, em_idx):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = np.mean(np.abs(em_est - act_est))
            
            sim_dict[act_name] = sim
            
            print(f'{em_name}과 {act_name}의 MAE : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    

    def sim_item_Cosine(self, em_idx):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = dot(em_est, act_est)/(norm(em_est)*norm(act_est))
            
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 Cosine 유사도 : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    

    def sim_item_Pearson(self, em_idx):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = pearson_similarity(em_est, act_est)
            
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 피어슨 유사도 : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), reverse=True ,key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    

    def sim_item_callback(self, em_idx,callback,reverse):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        sim_dict = {}
        for idx in self.action_df.index:
            act_name , *act_est = self.action_df.iloc[idx]
            act_est = np.array(act_est)
            sim = callback(em_est, act_est)
            
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 callback 유사도 : {sim}')
        
        sorted_dict = sorted(sim_dict.items(), reverse=reverse ,key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    


    def max_item_MAE(self, em_idx, n):
        em_name , * em_est = self.emothon_df.iloc[em_idx].tolist()
        em_est = np.array(em_est)
        effs = ['에너지','회복','순환','정화']
        em_dict = {}
        i = 0
        for eff in effs:
            em_dict[eff] = em_est[i]
            i = i + 1
        max_dict = sorted(em_dict.items(), reverse=True,key= lambda x:x[1])[:n]

        sort_em_list = []
        sort_em_est = []
        for x, y in max_dict:
            sort_em_list.append(x)
            sort_em_est.append(y)

        sort_em_df = self.emothon_df[sort_em_list] 
        em_est = np.array(sort_em_df.iloc[em_idx])
        sort_act_df = self.action_df[sort_em_list]
        print(f'{em_name}의 MAX 차원 {sort_em_list}')


        sim_dict = {}
        for idx in sort_act_df.index:
            act_name , *_ = self.action_df.iloc[idx]
            act_est = np.array(sort_act_df.iloc[idx])
            sim = np.mean(np.abs(em_est - act_est))
            sim_dict[act_name] = sim
            print(f'{em_name}과 {act_name}의 MAE : {sim}')
        sorted_dict = sorted(sim_dict.items(), key= lambda x:x[1])
        print("-"*20)
        print('{}에서 가장 유사한 item은 {}'.format(em_name,sorted_dict[0][0]))    

            

    def complete_item(self, em_idx, n):
        pass

In [192]:
# 감정과 가장 유사한 item을 추천 using MAE
XYZ = emotion2item(action_est, emotion_est)
for i in range(0,5):
    XYZ.sim_item_MAE(i)
    print('\n')


행복과 명상의 MAE : 0.11453574895858767
행복과 운동의 MAE : 0.05405441919962567
행복과 산책의 MAE : 0.022419507304827377
행복과 수면의 MAE : 0.05168657253185907
행복과 독서의 MAE : 0.024007445822159454
--------------------
행복에서 가장 유사한 item은 산책


열정과 명상의 MAE : 0.25271882116794586
열정과 운동의 MAE : 0.20978081723054248
열정과 산책의 MAE : 0.2532823234796524
열정과 수면의 MAE : 0.2772409841418267
열정과 독서의 MAE : 0.2668498965601126
--------------------
열정에서 가장 유사한 item은 운동


두려움과 명상의 MAE : 0.20936867408454418
두려움과 운동의 MAE : 0.1447985252986352
두려움과 산책의 MAE : 0.10948498360812665
두려움과 수면의 MAE : 0.11262693069875239
두려움과 독서의 MAE : 0.11047627218067646
--------------------
두려움에서 가장 유사한 item은 산책


분노과 명상의 MAE : 0.0986365601420403
분노과 운동의 MAE : 0.04553483178218208
분노과 산책의 MAE : 0.04734467715024948
분노과 수면의 MAE : 0.04420273005962373
분노과 독서의 MAE : 0.0730207102994124
--------------------
분노에서 가장 유사한 item은 수면


우울과 명상의 MAE : 0.24421456456184387
우울과 운동의 MAE : 0.1772751708825429
우울과 산책의 MAE : 0.1530719747145971
우울과 수면의 MAE : 0.16658060997724533
우울과 독서의 

In [194]:
# 감정과 가장 유사한 item을 추천 using 코사인 유사도
XYZ = emotion2item(action_est, emotion_est)
for i in range(0,5):
    XYZ.sim_item_Cosine(i)
    print('\n')


행복과 명상의 Cosine 유사도 : 0.9555365344130426
행복과 운동의 Cosine 유사도 : 0.9948724097369637
행복과 산책의 Cosine 유사도 : 0.997402612130226
행복과 수면의 Cosine 유사도 : 0.9872293553316945
행복과 독서의 Cosine 유사도 : 0.9974811565711201
--------------------
행복에서 가장 유사한 item은 명상


열정과 명상의 Cosine 유사도 : 0.8841794008093903
열정과 운동의 Cosine 유사도 : 0.9190057408679259
열정과 산책의 Cosine 유사도 : 0.9155783669871443
열정과 수면의 Cosine 유사도 : 0.898752858379603
열정과 독서의 Cosine 유사도 : 0.9215842543606314
--------------------
열정에서 가장 유사한 item은 명상


두려움과 명상의 Cosine 유사도 : 0.7662237089436588
두려움과 운동의 Cosine 유사도 : 0.8736731983980385
두려움과 산책의 Cosine 유사도 : 0.9216418960833406
두려움과 수면의 Cosine 유사도 : 0.9274185329797504
두려움과 독서의 Cosine 유사도 : 0.9171433721101699
--------------------
두려움에서 가장 유사한 item은 명상


분노과 명상의 Cosine 유사도 : 0.9652905303023102
분노과 운동의 Cosine 유사도 : 0.9925318536304438
분노과 산책의 Cosine 유사도 : 0.9918665974932611
분노과 수면의 Cosine 유사도 : 0.991353816834374
분노과 독서의 Cosine 유사도 : 0.9778332858258577
--------------------
분노에서 가장 유사한 item은 명상


우울과 명상의 Cosine 유사도 : 

In [195]:
# 감정과 가장 유사한 item을 추천 using 피어슨 유사도
XYZ = emotion2item(action_est, emotion_est)
for i in range(0,5):
    XYZ.sim_item_Pearson(i)
    print('\n')


행복과 명상의 피어슨 유사도 : 0.0792590684533288
행복과 운동의 피어슨 유사도 : 0.9403273934675496
행복과 산책의 피어슨 유사도 : 0.951067179850685
행복과 수면의 피어슨 유사도 : 0.7262368891908471
행복과 독서의 피어슨 유사도 : 0.9749145351948129
--------------------
행복에서 가장 유사한 item은 독서


열정과 명상의 피어슨 유사도 : -0.24547860315638195
열정과 운동의 피어슨 유사도 : 0.07329961517846129
열정과 산책의 피어슨 유사도 : 0.1181775231119519
열정과 수면의 피어슨 유사도 : -0.10162148591533239
열정과 독서의 피어슨 유사도 : 0.2995332271573808
--------------------
열정에서 가장 유사한 item은 독서


두려움과 명상의 피어슨 유사도 : -0.4293822268232103
두려움과 운동의 피어슨 유사도 : 0.5797929452505899
두려움과 산책의 피어슨 유사도 : 0.9397302384150749
두려움과 수면의 피어슨 유사도 : 0.9893585265404906
두려움과 독서의 피어슨 유사도 : 0.7598334785148667
--------------------
두려움에서 가장 유사한 item은 수면


분노과 명상의 피어슨 유사도 : -0.5159929813272701
분노과 운동의 피어슨 유사도 : 0.5757113124761282
분노과 산책의 피어슨 유사도 : 0.9478395653110442
분노과 수면의 피어슨 유사도 : 0.9230323957089217
분노과 독서의 피어슨 유사도 : 0.8350512586645834
--------------------
분노에서 가장 유사한 item은 산책


우울과 명상의 피어슨 유사도 : -0.05959399286846013
우울과 운동의 피어슨 유사도 : 0.8779272580498

In [196]:
# 추청치가 가장 높은 차원 1개에 대해서 MAE를 이용해서 추천
XYZ = emotion2item(action_est, emotion_est)
for idx in range(0,5):
    XYZ.max_item_MAE(idx,1)
    print("\n\n")

행복의 MAX 차원 ['회복']
행복과 명상의 MAE : 0.013684898614883423
행복과 운동의 MAE : 0.013990432024002075
행복과 산책의 MAE : 0.008198291063308716
행복과 수면의 MAE : 0.06239950656890869
행복과 독서의 MAE : 0.01601862907409668
--------------------
행복에서 가장 유사한 item은 산책



열정의 MAX 차원 ['에너지']
열정과 명상의 MAE : 0.5992499887943268
열정과 운동의 MAE : 0.635127067565918
열정과 산책의 MAE : 0.6796550850073497
열정과 수면의 MAE : 0.7182592749595642
열정과 독서의 MAE : 0.6736544668674469
--------------------
열정에서 가장 유사한 item은 명상



두려움의 MAX 차원 ['회복']
두려움과 명상의 MAE : 0.02186253666877741
두려움과 운동의 MAE : 0.005812793970108088
두려움과 산책의 MAE : 0.016375929117202703
두려움과 수면의 MAE : 0.07057714462280268
두려움과 독서의 MAE : 0.007840991020202692
--------------------
두려움에서 가장 유사한 item은 운동



분노의 MAX 차원 ['회복']
분노과 명상의 MAE : 0.050156980752945
분노과 운동의 MAE : 0.0778323113918305
분노과 산책의 MAE : 0.05564358830451971
분노과 수면의 MAE : 0.0014423727989197332
분노과 독서의 MAE : 0.0798605084419251
--------------------
분노에서 가장 유사한 item은 수면



우울의 MAX 차원 ['회복']
우울과 명상의 MAE : 0.5442720353603363
우울과 운동의 MAE

In [197]:
# 추청치가 가장 높은 차원 2개에 대해서 MAE를 이용해서 추천
XYZ = emotion2item(action_est, emotion_est)
for idx in range(0,5):
    XYZ.max_item_MAE(idx,2)
    print("\n\n")

행복의 MAX 차원 ['회복', '에너지']
행복과 명상의 MAE : 0.057028159499168424
행복과 운동의 MAE : 0.03924238681793213
행복과 산책의 MAE : 0.0140823076168696
행복과 수면의 MAE : 0.04051868617534635
행복과 독서의 MAE : 0.020992785692214994
--------------------
행복에서 가장 유사한 item은 산책



열정의 MAX 차원 ['에너지', '회복']
열정과 명상의 MAE : 0.37106049060821533
열정과 운동의 MAE : 0.3751613646745682
열정과 산책의 MAE : 0.40851973493893945
열정과 수면의 MAE : 0.4549224376678467
열정과 독서의 MAE : 0.39341096580028534
--------------------
열정에서 가장 유사한 item은 명상



두려움의 MAX 차원 ['회복', '정화']
두려움과 명상의 MAE : 0.023090407252311707
두려움과 운동의 MAE : 0.014528383811314965
두려움과 산책의 MAE : 0.022121593356132507
두려움과 수면의 MAE : 0.06451376775900519
두려움과 독서의 MAE : 0.02666732172171274
--------------------
두려움에서 가장 유사한 item은 운동



분노의 MAX 차원 ['회복', '정화']
분노과 명상의 MAE : 0.04609754681587222
분노과 운동의 MAE : 0.06047236422697705
분노과 산책의 MAE : 0.04706636071205142
분노과 수면의 MAE : 0.004674186309178735
분노과 독서의 MAE : 0.09585527578989667
--------------------
분노에서 가장 유사한 item은 수면



우울의 MAX 차원 ['회복', '정화']
우울과 명상의 

In [198]:
# 추청치가 가장 높은 차원 3개에 대해서 MAE를 이용해서 추천
XYZ = emotion2item(action_est, emotion_est)
for idx in range(0,5):
    XYZ.max_item_MAE(idx,3)
    print("\n\n")

행복의 MAX 차원 ['회복', '에너지', '순환']
행복과 명상의 MAE : 0.13857115308443707
행복과 운동의 MAE : 0.058287481466929116
행복과 산책의 MAE : 0.014566504293017904
행복과 수면의 MAE : 0.04339488016234503
행복과 독서의 MAE : 0.02288246320353617
--------------------
행복에서 가장 유사한 item은 산책



열정의 MAX 차원 ['에너지', '회복', '정화']
열정과 명상의 MAE : 0.27257383863131207
열정과 운동의 MAE : 0.27566585606998867
열정과 산책의 MAE : 0.296363674932056
열정과 수면의 MAE : 0.3171044323179457
열정과 독서의 MAE : 0.310744798845715
--------------------
열정에서 가장 유사한 item은 명상



두려움의 MAX 차원 ['회복', '정화', '에너지']
두려움과 명상의 MAE : 0.10201799372831981
두려움과 운동의 MAE : 0.08435095184379156
두려움과 산책의 MAE : 0.07457041905985937
두려움과 수면의 MAE : 0.08996380534436965
두려움과 독서의 MAE : 0.07960111068354712
--------------------
두려움에서 가장 유사한 item은 산책



분노의 MAX 차원 ['회복', '정화', '에너지']
분노과 명상의 MAE : 0.04294511675834659
분노과 운동의 MAE : 0.04056930210855273
분노과 산책의 MAE : 0.045965853664610146
분노과 수면의 MAE : 0.03057246738009985
분노과 독서의 MAE : 0.07649159100320603
--------------------
분노에서 가장 유사한 item은 수면



우울의 MAX 차원 

In [199]:
# 추청치가 가장 높은 차원 4개에 대해서 MAE를 이용해서 추천
XYZ = emotion2item(action_est, emotion_est)
for idx in range(0,5):
    XYZ.max_item_MAE(idx,1)
    print("\n\n")

행복의 MAX 차원 ['회복']
행복과 명상의 MAE : 0.013684898614883423
행복과 운동의 MAE : 0.013990432024002075
행복과 산책의 MAE : 0.008198291063308716
행복과 수면의 MAE : 0.06239950656890869
행복과 독서의 MAE : 0.01601862907409668
--------------------
행복에서 가장 유사한 item은 산책



열정의 MAX 차원 ['에너지']
열정과 명상의 MAE : 0.5992499887943268
열정과 운동의 MAE : 0.635127067565918
열정과 산책의 MAE : 0.6796550850073497
열정과 수면의 MAE : 0.7182592749595642
열정과 독서의 MAE : 0.6736544668674469
--------------------
열정에서 가장 유사한 item은 명상



두려움의 MAX 차원 ['회복']
두려움과 명상의 MAE : 0.02186253666877741
두려움과 운동의 MAE : 0.005812793970108088
두려움과 산책의 MAE : 0.016375929117202703
두려움과 수면의 MAE : 0.07057714462280268
두려움과 독서의 MAE : 0.007840991020202692
--------------------
두려움에서 가장 유사한 item은 운동



분노의 MAX 차원 ['회복']
분노과 명상의 MAE : 0.050156980752945
분노과 운동의 MAE : 0.0778323113918305
분노과 산책의 MAE : 0.05564358830451971
분노과 수면의 MAE : 0.0014423727989197332
분노과 독서의 MAE : 0.0798605084419251
--------------------
분노에서 가장 유사한 item은 수면



우울의 MAX 차원 ['회복']
우울과 명상의 MAE : 0.5442720353603363
우울과 운동의 MAE